**'top_bottom_value_counts_combined_PT_FT' Notebook:**
Historical Progression data along with their predicted outcome rate of progression is sorted  in ascending order by rate. We are manually checking the categories with their counts influencing least progression rate (top 30,100,200) and maximum  progression rate (bottom 30,100,200). Thus conclude by observing which categories of characteristics are mainly influential to Progression of students.

**Installing required libraries**

In [51]:
!pip install pycaret
!pip install scikit-optimize

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 53, Finished, Available, Finished)

**Importing required libraries/modules**

In [52]:
import pycaret
import numpy as np
import pandas as pd
import random
import json
import os
import logging
import cloudpickle
import matplotlib.pyplot as plt
from pycaret.classification import *
from sklearn.metrics import confusion_matrix,classification_report, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error,precision_score,roc_auc_score,accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import OrdinalEncoder
import pickle
from datetime import date
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from collections import Counter
from imblearn.combine import SMOTEENN
from datetime import datetime 
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from lime import lime_tabular 
from lime.lime_tabular import LimeTabularExplainer
from datetime import datetime 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import joblib
import skopt
import shutil
import shap
from enum import Enum
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from enum import Enum
import scipy.stats as stats
import pickle
from datetime import datetime
import joblib
import json
from datetime import datetime 
from dateutil.relativedelta import relativedelta
import pandas as pd
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import MapType, StringType

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 54, Finished, Available, Finished)

**Importing progression data from silver layer to do PT analysis**

In [53]:
df_p = spark.sql("SELECT * FROM SilverData.external_progression_data")
df_pt=df_p.toPandas()
df_pt

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 55, Finished, Available, Finished)

,Adult_HE_quintile,Age_group,Disability,Ethnicity,IDACI,IMD,Local_or_distance_learner,Sex,Predicted_outcome_rate,Derived_progression_quintile,createdAt
0,-- Any quintile --,21-25,Cognitive or learning difficulties,Asian or Asian British - Pakistani,-- Any quintile --,-- Any quintile --,Local,Female,0.620000,1,2024-11-25 08:59:18.184056
1,-- Any quintile --,21-25,Cognitive or learning difficulties,Black or black British - African,-- Any quintile --,-- Any quintile --,Local,Female,0.730000,1,2024-11-25 08:59:18.184056
2,-- Any quintile --,21-25,Cognitive or learning difficulties,Black or black British - Caribbean,-- Any quintile --,-- Any quintile --,Local,Female,0.670000,1,2024-11-25 08:59:18.184056
3,-- Any quintile --,21-25,Cognitive or learning difficulties,Mixed - other,-- Any quintile --,-- Any quintile --,Local,Female,0.740000,1,2024-11-25 08:59:18.184056
4,-- Any quintile --,21-25,Cognitive or learning difficulties,Mixed - other,-- Any quintile --,-- Any quintile --,Local,Female,0.740000,1,2024-11-25 08:59:18.184056
...,...,...,...,...,...,...,...,...,...,...,...
770852,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770853,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770854,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770855,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056


**Mapping Any categories as blanks and Local as FT and Distance as PT**

In [54]:
df_pt1 = df_pt.replace({
    '-- Any quintile --':'', 
    '-- Any disability --':'', 
    '-- Any ethnicity --':'', 
    '-- Any quintile --':'', 
    '-- Any learner --': '',
    '-- Any sex --': '',
    'Distance':'PT',
    '-- Any age group --':''  
})

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 56, Finished, Available, Finished)

**Selecting PT combinaation only**

In [55]:
df_pt1= df_pt1[df_pt1['Local_or_distance_learner']=='PT']
df_pt1

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 57, Finished, Available, Finished)

,Adult_HE_quintile,Age_group,Disability,Ethnicity,IDACI,IMD,Local_or_distance_learner,Sex,Predicted_outcome_rate,Derived_progression_quintile,createdAt
1003,,21-25,Cognitive or learning difficulties,Mixed - white and black African,,,PT,Female,0.750000,1,2024-11-25 08:59:18.184056
1004,Quintile 1,21-25,Cognitive or learning difficulties,,,,PT,Female,0.720000,1,2024-11-25 08:59:18.184056
1005,,21-25,Cognitive or learning difficulties,,Quintile 1,,PT,Female,0.740000,1,2024-11-25 08:59:18.184056
1006,,21-25,Cognitive or learning difficulties,,Quintile 1,,PT,Female,0.740000,1,2024-11-25 08:59:18.184056
1007,,21-25,Cognitive or learning difficulties,,,Quintile 1,PT,Female,0.730000,1,2024-11-25 08:59:18.184056
...,...,...,...,...,...,...,...,...,...,...,...
770847,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770848,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770849,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056
770850,,,No disability reported,Asian or Asian British - Indian,,,PT,Male,0.820000,3,2024-11-25 08:59:18.184056


**Created a Concatenated column by adding all the characteristics- Adult he quintile, age group, disability, ethnicity, idaci, imd, local or distance learner, sex**


In [56]:
# Concatenate columns by converting all values to strings
df_pt1['Concatenated_column'] = (
    df_pt1['Adult_HE_quintile'].astype(str) + 
    df_pt1['Age_group'].astype(str) + 
    df_pt1['Disability'].astype(str) + 
    df_pt1['Ethnicity'].astype(str) + 
    df_pt1['IDACI'].astype(str) + 
    df_pt1['IMD'].astype(str) + 
    df_pt1['Local_or_distance_learner'].astype(str) + 
    df_pt1['Sex'].astype(str)
)


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 58, Finished, Available, Finished)

**Selecting some necessary columns and saving as a dataframe**

In [57]:
df_pt1=df_pt1[['Predicted_outcome_rate','Derived_progression_quintile','Concatenated_column']]

df_pt1

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 59, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column
1003,0.750000,1,21-25Cognitive or learning difficultiesMixed -...
1004,0.720000,1,Quintile 121-25Cognitive or learning difficult...
1005,0.740000,1,21-25Cognitive or learning difficultiesQuintil...
1006,0.740000,1,21-25Cognitive or learning difficultiesQuintil...
1007,0.730000,1,21-25Cognitive or learning difficultiesQuintil...
...,...,...,...
770847,0.820000,3,No disability reportedAsian or Asian British -...
770848,0.820000,3,No disability reportedAsian or Asian British -...
770849,0.820000,3,No disability reportedAsian or Asian British -...
770850,0.820000,3,No disability reportedAsian or Asian British -...


**Fetching student characteristics table from silver layer**

In [58]:
df_char = spark.sql("SELECT * FROM SilverData.event_student_characteristics")
df_characteristics=df_char.toPandas()
#renaming 'stunumber' with 'StudentNumber'
df_characteristics=df_characteristics.rename(columns={'stunumber':'StudentNumber'}, inplace=False)
# Remove columns that end with 'value'
df_characteristics = df_characteristics[[col for col in df_characteristics.columns if not col.endswith('value')]]
#dropping some columns('student_id','created_at','modified_at') as those are not required
df_characteristics=df_characteristics.drop(columns=['student_id','created_at','modified_at'])
df_characteristics

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 60, Finished, Available, Finished)

,StudentNumber,age_on_entry,ethnicity_16_groups,sex,ethnicity_6_groups,gender_identity,tundra_msoa_quintile_desc,adult_he_2011_quintile_desc,polar4_quintile_desc,imd_desc,idaci_desc,religion_or_belief,sexual_orientation,parental_education,disability_type,disability_status,care_leaver,entry_qualification_desc
0,STU223216,Under 21 years on entry,Refused_or_Unknown,Male,Refused_unknown_or_not_collected,Yes,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Hindu,Heterosexual,Yes,No_disability_reported_or_unknown,No_Disability_Reported,"Refused, unknown or not collected",Other qualification at level 3
1,STU233638,21 to 25 years on entry,Refused_or_Unknown,Male,Refused_unknown_or_not_collected,Yes,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Hindu,Other_Sexual_Orientation,Not_known,No_disability_reported_or_unknown,No_Disability_Reported,Care_experienced,Other qualification at level 3
2,STU232679,21 to 25 years on entry,Refused_or_Unknown,Male,Refused_unknown_or_not_collected,Yes,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Hindu,Refused_Unknown_or_not_collected,Yes,No_disability_reported_or_unknown,No_Disability_Reported,"Refused, unknown or not collected",Other qualification at level 3
3,STU226091,21 to 25 years on entry,Refused_or_Unknown,Female,Refused_unknown_or_not_collected,Information refused,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Hindu,Refused_Unknown_or_not_collected,Yes,No_disability_reported_or_unknown,No_Disability_Reported,"Refused, unknown or not collected",Other qualification at level 3
4,STU226874,Under 21 years on entry,Refused_or_Unknown,Male,Refused_unknown_or_not_collected,No,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Unknown or invalid home postcode,Hindu,Refused_Unknown_or_not_collected,Not_known,No_disability_reported_or_unknown,No_Disability_Reported,"Refused, unknown or not collected",Other qualification at level 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96210,STU113403,31 to 40 years on entry,White,Female,White,Yes,Quintile 5,Quintile 3,Quintile 4,Quintile 2 England,Quintile 2,Christian,Heterosexual,No,No_disability_reported_or_unknown,No_Disability_Reported,Care_experienced,Higher National Diploma (HND)
96211,STU146681,31 to 40 years on entry,White,Female,White,Yes,Quintile 5,Quintile 3,Quintile 4,Quintile 2 England,Quintile 2,Christian,Heterosexual,No,No_disability_reported_or_unknown,No_Disability_Reported,Care_experienced,Other qualification at level 3
96212,STU152040,31 to 40 years on entry,White,Female,White,Yes,Quintile 5,Quintile 3,Quintile 4,Quintile 2 England,Quintile 2,Christian,Heterosexual,No,No_disability_reported_or_unknown,No_Disability_Reported,Care_experienced,UK first degree with honours
96213,STU129002,31 to 40 years on entry,White,Female,White,Yes,Quintile 5,Quintile 3,Quintile 4,Quintile 2 England,Quintile 2,Christian,Heterosexual,No,No_disability_reported_or_unknown,No_Disability_Reported,Care_experienced,Other qualification at level C


**Changing some mappings as per ABCS data**

In [59]:
df_characteristics = df_characteristics.replace({
    '51 years and over on entry':'51+', 
    '41 to 50 years on entry':'41-50', 
    '26 to 30 years on entry':'26-30', 
    'Under 21 years on entry':'20', 
    '21 to 25 years on entry': '21-25', 
    '31 to 40 years on entry': '31-40', 
    'Unknown':'',

    'Refused_or_Unknown':'Refused or unknown',
    
  
    'White':'White',
    'Black_or_black_British_African':'Black or black British - African',
    'Other_ethnic_group':'Other ethnic group', 
    'Mixed_other':'Mixed - other',
    'Gypsy_or_Traveller':'',
    'Black_or_black_British_Caribbean':'Black or black British - Caribbean',
    'Asian_or_Asian_British_Bangladeshi':'Asian or Asian British - Bangladeshi',
    'Asian_or_Asian_British_Pakistani':'Asian or Asian British - Pakistani',
    'Mixed_White_and_black_African':'Mixed - white and black African',
    'Mixed_White_and_black_Caribbean':'Mixed - white and black Caribbean',
    'Asian_or_Asian_British_other':'Asian or Asian British - other', 
    'Asian_or_Asian_British_Chinese':'Asian or Asian British - Chinese',
    'Mixed_White_and_Asian':'Mixed - white and Asian', 
    'Asian_or_Asian_British_Indian':'Asian or Asian British - Indian',
    
    'Refused_unknown_or_not_collected':'',
    'Other_sex':'',

    'Unknown or invalid home postcode':'',
    'Quintile 1 England':'Quintile 1', 
    'Quintile 3 England':'Quintile 3', 
    'Quintile 2 England':'Quintile 2',
    'Quintile 4 England':'Quintile 4', 
    'Quintile 5 England':'Quintile 5',

    'No_disability_reported_or_unknown':'No disability reported', 
    'Mental_health_condition':'Mental health condition',
    'Cognitive_or_learning_difficulties':'Cognitive or learning difficulties',
    'Sensory_medical_or_physical_impairment':'Sensory, medical or physical impairments',
    'Multiple_or_other_impairments':'Multiple or other impairments',
    'Social_or_communication_impairment':'Social or communication impairment'
    
})


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 61, Finished, Available, Finished)

**Fetching student_code, study mode details and event_description from event table and applied filter Event_Type= 'Progression Outcome'**

In [60]:
df_study_mode = spark.sql("Select Student_Code,Event_Context,Event_Description from event_students where Event_Type = 'Progression Outcome'")
df_study_mode_pandas = df_study_mode.toPandas()
df_study_mode_pandas['Study Mode Details'] = df_study_mode_pandas['Event_Context']
df_study_mode_pandas = df_study_mode_pandas[['Student_Code', 'Study Mode Details','Event_Description']]
df_study_mode_pandas

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 62, Finished, Available, Finished)

,Student_Code,Study Mode Details,Event_Description
0,STU37252,"{""Ipstartmode"":""PT"",""Iplevel"":""PGTM""}",Progression: Positive
1,STU64336,"{""Ipstartmode"":""PT"",""Iplevel"":""PGTM""}",Progression: Positive
2,STU51841,"{""Ipstartmode"":""PT"",""Iplevel"":""PGTM""}",Progression: Positive
3,STU58143,"{""Ipstartmode"":""PT"",""Iplevel"":""PGTM""}",Progression: Positive
4,STU37792,"{""Ipstartmode"":""PT"",""Iplevel"":""PGTM""}",Progression: Positive
...,...,...,...
1573,STU53397,"{""Ipstartmode"":""FT"",""Iplevel"":""DEG""}",Progression: Negative
1574,STU58274,"{""Ipstartmode"":""FT"",""Iplevel"":""DEG""}",Progression: Positive
1575,STU53864,"{""Ipstartmode"":""FT"",""Iplevel"":""DEG""}",Progression: Negative
1576,STU53853,"{""Ipstartmode"":""FT"",""Iplevel"":""DEG""}",Progression: Negative


**Fetching details like ipstartmode and iplevel**

In [61]:
# Define a schema as a MapType where both keys and values are of StringType
schema = MapType(StringType(), StringType())

# Convert a Pandas DataFrame (df_study_mode_pandas) to a Spark DataFrame
df_spark_study_mode = spark.createDataFrame(df_study_mode_pandas)

# Parse the JSON string in the 'Study Mode Details' column into a MapType using the defined schema
df_spark_study_mode = df_spark_study_mode.withColumn("Study Mode Details", from_json(col("Study Mode Details"), schema))

# Extract the value associated with the key 'Ipstartmode' from the 'Study Mode Details' map and create a new column 'IPSTARTMODE'
df_spark_study_mode = df_spark_study_mode.withColumn("IPSTARTMODE", col("Study Mode Details").getItem("Ipstartmode"))

# Extract the value associated with the key 'Iplevel' from the 'Study Mode Details' map and create a new column 'IPLEVEL'
df_spark_study_mode = df_spark_study_mode.withColumn("IPLEVEL", col("Study Mode Details").getItem("Iplevel"))

# Drop the original 'Study Mode Details' column as it's no longer needed
df_spark_study_mode=df_spark_study_mode.drop("Study Mode Details")

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 63, Finished, Available, Finished)

**Converting spark dataframe('df_spark_study_mode') into pandas dataframe**

In [62]:
df_progression_historical = df_spark_study_mode.toPandas()
df_progression_historical

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 64, Finished, Available, Finished)

,Student_Code,Event_Description,IPSTARTMODE,IPLEVEL
0,STU37252,Progression: Positive,PT,PGTM
1,STU64336,Progression: Positive,PT,PGTM
2,STU51841,Progression: Positive,PT,PGTM
3,STU58143,Progression: Positive,PT,PGTM
4,STU37792,Progression: Positive,PT,PGTM
...,...,...,...,...
1573,STU53397,Progression: Negative,FT,DEG
1574,STU58274,Progression: Positive,FT,DEG
1575,STU53864,Progression: Negative,FT,DEG
1576,STU53853,Progression: Negative,FT,DEG


**Fetching the actual progression flags for students**

In [63]:
df_progression_historical['Outcome'] = df_progression_historical['Event_Description'].str.split(': ').str[1]

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 65, Finished, Available, Finished)

**Dropping 'Event_Description' column from the dataframe**

In [64]:
df_hist = df_progression_historical.drop('Event_Description', axis=1)
df_hist

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 66, Finished, Available, Finished)

,Student_Code,IPSTARTMODE,IPLEVEL,Outcome
0,STU37252,PT,PGTM,Positive
1,STU64336,PT,PGTM,Positive
2,STU51841,PT,PGTM,Positive
3,STU58143,PT,PGTM,Positive
4,STU37792,PT,PGTM,Positive
...,...,...,...,...
1573,STU53397,FT,DEG,Negative
1574,STU58274,FT,DEG,Positive
1575,STU53864,FT,DEG,Negative
1576,STU53853,FT,DEG,Negative


**Selecting some columns for the dataframe('df_hist')**

In [65]:
df_hist.columns = ['StudentNumber', 'IPSTARTMODE', 'IPLEVEL', 'Outcome']

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 67, Finished, Available, Finished)

**Applying filter on 'PT' combination and deselecting 'Neutral' combination**

In [66]:
df_hist= df_hist[df_hist['IPSTARTMODE']=='PT']
df_hist= df_hist[df_hist['Outcome']!='Neutral']

df_hist

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 68, Finished, Available, Finished)

,StudentNumber,IPSTARTMODE,IPLEVEL,Outcome
0,STU37252,PT,PGTM,Positive
1,STU64336,PT,PGTM,Positive
2,STU51841,PT,PGTM,Positive
3,STU58143,PT,PGTM,Positive
4,STU37792,PT,PGTM,Positive
...,...,...,...,...
1195,STU43922,PT,DEG,Positive
1196,STU52911,PT,DEG,Positive
1197,STU53541,PT,DEG,Positive
1198,STU102116,PT,DEG,Positive


**Appyling inner join operation on historical data and characteristics data**

In [67]:
concat_hist_char = df_characteristics.merge(df_hist,on='StudentNumber', how='inner')
concat_hist_char = concat_hist_char.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 69, Finished, Available, Finished)

**Printing the dataframe**

In [68]:
concat_hist_char

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 70, Finished, Available, Finished)

,StudentNumber,age_on_entry,ethnicity_16_groups,sex,ethnicity_6_groups,gender_identity,tundra_msoa_quintile_desc,adult_he_2011_quintile_desc,polar4_quintile_desc,imd_desc,...,religion_or_belief,sexual_orientation,parental_education,disability_type,disability_status,care_leaver,entry_qualification_desc,IPSTARTMODE,IPLEVEL,Outcome
0,STU71842,26-30,White,,White,No,Quintile 3,Quintile 5,Quintile 3,Quintile 5,...,Any_other_religion_or_belief,Other_Sexual_Orientation,No,Mental health condition,Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM,Positive
1,STU66810,31-40,White,Female,White,Not available,Quintile 1,Quintile 2,Quintile 1,Quintile 3,...,Any_other_religion_or_belief,Heterosexual,Not_known,"Sensory, medical or physical impairments",Disability_Reported,"Refused, unknown or not collected",Other taught qualification at level M,PT,PGTM,Positive
2,STU24349,41-50,White,Female,White,Not available,Quintile 4,Quintile 5,Quintile 5,Quintile 4,...,,Refused_Unknown_or_not_collected,Not_known,"Sensory, medical or physical impairments",Disability_Reported,"Refused, unknown or not collected",Other qualification at level C,PT,DEG,Positive
3,STU56498,20,White,Female,White,Not available,Quintile 1,Quintile 1,Quintile 1,Quintile 1,...,,Heterosexual,Not_known,Mental health condition,Disability_Reported,"Refused, unknown or not collected",Higher National Diploma (HND),PT,DEG,Positive
4,STU62198,26-30,White,Male,White,Not available,Quintile 1,Quintile 1,Quintile 1,Quintile 1,...,,Refused_Unknown_or_not_collected,Not_known,"Sensory, medical or physical impairments",Disability_Reported,"Refused, unknown or not collected",UK first degree with honours,PT,PGTM,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,STU72329,31-40,White,Female,White,Yes,Quintile 2,Quintile 2,Quintile 2,Quintile 2,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM,Positive
750,STU46308,31-40,White,Female,White,Yes,Quintile 5,Quintile 5,Quintile 5,Quintile 3,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Level 3 qualifications of which none are subje...,PT,DEG,Positive
752,STU54442,31-40,White,Female,White,Yes,Quintile 3,Quintile 4,Quintile 4,Quintile 3,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Level 3 qualifications of which some are subje...,PT,DEG,Positive
753,STU80079,31-40,White,Female,White,Yes,Quintile 1,Quintile 1,Quintile 1,Quintile 2,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,UK first degree with honours,PT,PGTM,Positive


**Selecting relevant columns from the below dataframe**

In [69]:
concat_hist_char = concat_hist_char[['StudentNumber', 'age_on_entry', 'ethnicity_16_groups', 'sex',
       'adult_he_2011_quintile_desc', 'imd_desc',
       'idaci_desc',  'disability_type', 'IPLEVEL','IPSTARTMODE',
       'Outcome']]

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 71, Finished, Available, Finished)

**Creating copy of the below dataframe and saving in another dataframe**

In [70]:
concat_hist_char1 = concat_hist_char.copy()
concat_hist_char_export = concat_hist_char.copy()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 72, Finished, Available, Finished)

**Creating concatenated column by joining characteristics in same order as above to map with external progression data**

In [71]:
# Concatenate columns by converting all values to strings
concat_hist_char['Concatenated_column'] = (
    concat_hist_char['adult_he_2011_quintile_desc'].astype(str) + 
    concat_hist_char['age_on_entry'].astype(str) + 
    concat_hist_char['disability_type'].astype(str) + 
    concat_hist_char['ethnicity_16_groups'].astype(str) + 
    concat_hist_char['idaci_desc'].astype(str) + 
    concat_hist_char['imd_desc'].astype(str) + 
    concat_hist_char['IPSTARTMODE'].astype(str) + 
    concat_hist_char['sex'].astype(str)
)

#renaming col name and slecting some columns 
concat_hist_char.rename(columns={'Outcome': 'Actual_Outcome'}, inplace=True)
concat_hist_char=concat_hist_char[['StudentNumber','IPLEVEL','Actual_Outcome','Concatenated_column']]
concat_hist_char

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 73, Finished, Available, Finished)

,StudentNumber,IPLEVEL,Actual_Outcome,Concatenated_column
0,STU71842,PGTM,Positive,Quintile 526-30Mental health conditionWhiteQui...
1,STU66810,PGTM,Positive,"Quintile 231-40Sensory, medical or physical im..."
2,STU24349,DEG,Positive,"Quintile 541-50Sensory, medical or physical im..."
3,STU56498,DEG,Positive,Quintile 120Mental health conditionWhiteQuinti...
4,STU62198,PGTM,Positive,"Quintile 126-30Sensory, medical or physical im..."
...,...,...,...,...
749,STU72329,PGTM,Positive,Quintile 231-40No disability reportedWhiteQuin...
750,STU46308,DEG,Positive,Quintile 531-40No disability reportedWhiteQuin...
752,STU54442,DEG,Positive,Quintile 431-40No disability reportedWhiteQuin...
753,STU80079,PGTM,Positive,Quintile 131-40No disability reportedWhiteQuin...


In [72]:
#dropping duplicates from the dataframe
df_pt1 = df_pt1.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 74, Finished, Available, Finished)

**Inner joining with historical Arden data with external progression data on concatenated column**

In [73]:
joined_df = df_pt1.merge(concat_hist_char,on='Concatenated_column', how='inner')
joined_df = joined_df.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 75, Finished, Available, Finished)

In [74]:
#displaying the dataframe
joined_df

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 76, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column,StudentNumber,IPLEVEL,Actual_Outcome
0,0.730000,1,"Quintile 321-25Sensory, medical or physical im...",STU59079,PGTM,Positive
1,0.750000,1,"Quintile 221-25Sensory, medical or physical im...",STU59158,PGTM,Negative
2,0.660000,1,Quintile 321-25Social or communication impairm...,STU69756,PGTM,Positive
3,0.600000,1,Quintile 121-25Mental health conditionWhiteQui...,STU56579,DEG,Negative
4,0.530000,1,Quintile 221-25Mental health conditionAsian or...,STU74215,DEG,Negative
...,...,...,...,...,...,...
665,0.820000,3,Quintile 231-40No disability reportedOther eth...,STU54158,DEG,Positive
666,0.900000,5,Quintile 531-40No disability reportedAsian or ...,STU42083,DEG,Positive
667,0.890000,5,Quintile 531-40No disability reportedAsian or ...,STU49976,PGTM,Positive
668,0.860000,5,Quintile 531-40No disability reportedAsian or ...,STU40896,PGTM,Positive


**Applying inner join to tag predicted outcome rate and derive quintile to each student**

In [75]:
export_pt_df = pd.merge(joined_df,concat_hist_char1,on='StudentNumber',how = 'inner')

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 77, Finished, Available, Finished)

**Selecting relevant columns**

In [76]:
export_pt_df = export_pt_df[[ 'StudentNumber', 'IPLEVEL_x', 
       'age_on_entry', 'ethnicity_16_groups', 'sex',
       'adult_he_2011_quintile_desc', 'imd_desc', 'idaci_desc',
       'disability_type', 'IPSTARTMODE',
       'Concatenated_column','Actual_Outcome','Predicted_outcome_rate', 'Derived_progression_quintile']]

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 78, Finished, Available, Finished)

In [77]:
#displaying dataframe
export_pt_df

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 79, Finished, Available, Finished)

,StudentNumber,IPLEVEL_x,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPSTARTMODE,Concatenated_column,Actual_Outcome,Predicted_outcome_rate,Derived_progression_quintile
0,STU59079,PGTM,21-25,White,Female,Quintile 3,Quintile 2,Quintile 4,"Sensory, medical or physical impairments",PT,"Quintile 321-25Sensory, medical or physical im...",Positive,0.730000,1
1,STU59158,PGTM,21-25,White,Female,Quintile 2,Quintile 3,Quintile 4,"Sensory, medical or physical impairments",PT,"Quintile 221-25Sensory, medical or physical im...",Negative,0.750000,1
2,STU69756,PGTM,21-25,White,Female,Quintile 3,Quintile 2,Quintile 2,Social or communication impairment,PT,Quintile 321-25Social or communication impairm...,Positive,0.660000,1
3,STU56579,DEG,21-25,White,Female,Quintile 1,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 121-25Mental health conditionWhiteQui...,Negative,0.600000,1
4,STU74215,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 2,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 221-25Mental health conditionAsian or...,Negative,0.530000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,STU54158,DEG,31-40,Other ethnic group,Male,Quintile 2,Quintile 3,Quintile 4,No disability reported,PT,Quintile 231-40No disability reportedOther eth...,Positive,0.820000,3
682,STU42083,DEG,31-40,Asian or Asian British - Indian,Male,Quintile 5,Quintile 5,Quintile 5,No disability reported,PT,Quintile 531-40No disability reportedAsian or ...,Positive,0.900000,5
683,STU49976,PGTM,31-40,Asian or Asian British - Indian,Male,Quintile 5,Quintile 5,Quintile 4,No disability reported,PT,Quintile 531-40No disability reportedAsian or ...,Positive,0.890000,5
684,STU40896,PGTM,31-40,Asian or Asian British - Indian,Male,Quintile 5,Quintile 4,Quintile 2,No disability reported,PT,Quintile 531-40No disability reportedAsian or ...,Positive,0.860000,5


**Creating copy of the below dataframe and saving in another dataframe**

In [78]:
raw_data_pt=export_pt_df.copy()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 80, Finished, Available, Finished)

**Importing progression data again to do FT analysis**

In [79]:
df_ft_p = spark.sql("SELECT * FROM SilverData.external_progression_data")
df_ft=df_ft_p.toPandas()
df_ft

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 81, Finished, Available, Finished)

,Adult_HE_quintile,Age_group,Disability,Ethnicity,IDACI,IMD,Local_or_distance_learner,Sex,Predicted_outcome_rate,Derived_progression_quintile,createdAt
0,-- Any quintile --,21-25,Cognitive or learning difficulties,Asian or Asian British - Pakistani,-- Any quintile --,-- Any quintile --,Local,Female,0.620000,1,2024-11-25 08:59:18.184056
1,-- Any quintile --,21-25,Cognitive or learning difficulties,Black or black British - African,-- Any quintile --,-- Any quintile --,Local,Female,0.730000,1,2024-11-25 08:59:18.184056
2,-- Any quintile --,21-25,Cognitive or learning difficulties,Black or black British - Caribbean,-- Any quintile --,-- Any quintile --,Local,Female,0.670000,1,2024-11-25 08:59:18.184056
3,-- Any quintile --,21-25,Cognitive or learning difficulties,Mixed - other,-- Any quintile --,-- Any quintile --,Local,Female,0.740000,1,2024-11-25 08:59:18.184056
4,-- Any quintile --,21-25,Cognitive or learning difficulties,Mixed - other,-- Any quintile --,-- Any quintile --,Local,Female,0.740000,1,2024-11-25 08:59:18.184056
...,...,...,...,...,...,...,...,...,...,...,...
770852,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770853,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770854,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056
770855,-- Any quintile --,-- Any age group --,No disability reported,Asian or Asian British - Indian,-- Any quintile --,-- Any quintile --,Neither,Male,0.820000,3,2024-11-25 08:59:18.184056


**Mapping Any categories as blanks and Local as FT and Distance as PT**

In [80]:
df_ft1 = df_ft.replace({
    '-- Any quintile --':'', 
    '-- Any disability --':'', 
    '-- Any ethnicity --':'', 
    '-- Any quintile --':'', 
    '-- Any learner --': '',
    '-- Any sex --': '',
    'Local':'FT',
    'Distance':'PT',
    '-- Any age group --':''  
})

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 82, Finished, Available, Finished)

**Selecting FT combination only**

In [81]:
df_ft1= df_ft1[df_ft1['Local_or_distance_learner']=='FT']
df_ft1

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 83, Finished, Available, Finished)

,Adult_HE_quintile,Age_group,Disability,Ethnicity,IDACI,IMD,Local_or_distance_learner,Sex,Predicted_outcome_rate,Derived_progression_quintile,createdAt
0,,21-25,Cognitive or learning difficulties,Asian or Asian British - Pakistani,,,FT,Female,0.620000,1,2024-11-25 08:59:18.184056
1,,21-25,Cognitive or learning difficulties,Black or black British - African,,,FT,Female,0.730000,1,2024-11-25 08:59:18.184056
2,,21-25,Cognitive or learning difficulties,Black or black British - Caribbean,,,FT,Female,0.670000,1,2024-11-25 08:59:18.184056
3,,21-25,Cognitive or learning difficulties,Mixed - other,,,FT,Female,0.740000,1,2024-11-25 08:59:18.184056
4,,21-25,Cognitive or learning difficulties,Mixed - other,,,FT,Female,0.740000,1,2024-11-25 08:59:18.184056
...,...,...,...,...,...,...,...,...,...,...,...
770837,,,No disability reported,Mixed - white and black Caribbean,,,FT,Male,0.820000,3,2024-11-25 08:59:18.184056
770838,,,No disability reported,Mixed - white and black Caribbean,,,FT,Male,0.820000,3,2024-11-25 08:59:18.184056
770839,,,No disability reported,Mixed - white and black Caribbean,,,FT,Male,0.820000,3,2024-11-25 08:59:18.184056
770840,,,No disability reported,Mixed - white and black Caribbean,,,FT,Male,0.820000,3,2024-11-25 08:59:18.184056


**Created a Concatenated column by adding all the characteristics- Adult he quintile, age group, disability, ethnicity, idaci, imd, local or distance learner, sex**

In [82]:
# Concatenate columns by converting all values to strings
df_ft1['Concatenated_column'] = (
    df_ft1['Adult_HE_quintile'].astype(str) + 
    df_ft1['Age_group'].astype(str) + 
    df_ft1['Disability'].astype(str) + 
    df_ft1['Ethnicity'].astype(str) + 
    df_ft1['IDACI'].astype(str) + 
    df_ft1['IMD'].astype(str) + 
    df_ft1['Local_or_distance_learner'].astype(str) + 
    df_ft1['Sex'].astype(str)
)


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 84, Finished, Available, Finished)

**Selecting some necessary columns and saving as a dataframe**

In [83]:
df_ft1=df_ft1[['Predicted_outcome_rate','Derived_progression_quintile','Concatenated_column']]

df_ft1

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 85, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column
0,0.620000,1,21-25Cognitive or learning difficultiesAsian o...
1,0.730000,1,21-25Cognitive or learning difficultiesBlack o...
2,0.670000,1,21-25Cognitive or learning difficultiesBlack o...
3,0.740000,1,21-25Cognitive or learning difficultiesMixed -...
4,0.740000,1,21-25Cognitive or learning difficultiesMixed -...
...,...,...,...
770837,0.820000,3,No disability reportedMixed - white and black ...
770838,0.820000,3,No disability reportedMixed - white and black ...
770839,0.820000,3,No disability reportedMixed - white and black ...
770840,0.820000,3,No disability reportedMixed - white and black ...


**Converting spark dataframe('df_spark_study_mode') into pandas dataframe**

In [84]:
df_progression_historical_f = df_spark_study_mode.toPandas()
df_progression_historical_f

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 86, Finished, Available, Finished)

,Student_Code,Event_Description,IPSTARTMODE,IPLEVEL
0,STU37252,Progression: Positive,PT,PGTM
1,STU64336,Progression: Positive,PT,PGTM
2,STU51841,Progression: Positive,PT,PGTM
3,STU58143,Progression: Positive,PT,PGTM
4,STU37792,Progression: Positive,PT,PGTM
...,...,...,...,...
1573,STU53397,Progression: Negative,FT,DEG
1574,STU58274,Progression: Positive,FT,DEG
1575,STU53864,Progression: Negative,FT,DEG
1576,STU53853,Progression: Negative,FT,DEG


**Fetching the actual progression flags for students**

In [85]:
df_progression_historical_f['Outcome'] = df_progression_historical_f['Event_Description'].str.split(': ').str[1]

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 87, Finished, Available, Finished)

**Dropping 'Event_Description' column from the dataframe**

In [86]:
df_hist_f = df_progression_historical_f.drop('Event_Description', axis=1)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 88, Finished, Available, Finished)

**Selecting some columns for the dataframe('df_hist_f')**

In [87]:
df_hist_f.columns = ['StudentNumber', 'IPSTARTMODE', 'IPLEVEL', 'Outcome']

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 89, Finished, Available, Finished)

**Applying filter on 'FT' combination and deselecting 'Neutral' combination**

In [88]:
df_hist_f= df_hist_f[df_hist_f['IPSTARTMODE']=='FT']
df_hist_f= df_hist_f[df_hist_f['Outcome']!='Neutral']

df_hist_f

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 90, Finished, Available, Finished)

,StudentNumber,IPSTARTMODE,IPLEVEL,Outcome
143,STU51724,FT,PGTM,Negative
421,STU70305,FT,DEG,Negative
422,STU62921,FT,DEG,Negative
423,STU64009,FT,DEG,Negative
424,STU46216,FT,DEG,Negative
...,...,...,...,...
1573,STU53397,FT,DEG,Negative
1574,STU58274,FT,DEG,Positive
1575,STU53864,FT,DEG,Negative
1576,STU53853,FT,DEG,Negative


**Appyling inner join operation on historical data and characteristics data**

In [89]:
concat_hist_char_f = df_characteristics.merge(df_hist_f,on='StudentNumber', how='inner')
concat_hist_char_f = concat_hist_char_f.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 91, Finished, Available, Finished)

**Displaying the dataframe**

In [90]:
concat_hist_char_f

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 92, Finished, Available, Finished)

,StudentNumber,age_on_entry,ethnicity_16_groups,sex,ethnicity_6_groups,gender_identity,tundra_msoa_quintile_desc,adult_he_2011_quintile_desc,polar4_quintile_desc,imd_desc,...,religion_or_belief,sexual_orientation,parental_education,disability_type,disability_status,care_leaver,entry_qualification_desc,IPSTARTMODE,IPLEVEL,Outcome
0,STU69375,41-50,White,Male,White,Yes,Quintile 1,Quintile 2,Quintile 1,Quintile 1,...,Buddhist,Heterosexual,No,"Sensory, medical or physical impairments",Disability_Reported,Care_experienced,UK first degree with honours,FT,DEG,Negative
1,STU68962,41-50,White,Male,White,Not available,Quintile 5,Quintile 5,Quintile 4,Quintile 3,...,Any_other_religion_or_belief,Heterosexual,Not_known,Mental health condition,Disability_Reported,"Refused, unknown or not collected",Higher National Diploma (HND),FT,DEG,Negative
2,STU65338,31-40,White,Female,White,Yes,Quintile 3,Quintile 4,Quintile 4,Quintile 5,...,Any_other_religion_or_belief,Heterosexual,Yes,Mental health condition,Disability_Reported,Care_experienced,Level 3 qualifications of which some are subje...,FT,DEG,Positive
3,STU62316,26-30,White,Female,White,Yes,Quintile 2,Quintile 1,Quintile 2,Quintile 2,...,Any_other_religion_or_belief,Lesbian_Gay_OR_Bisexual,No,Mental health condition,Disability_Reported,Care_experienced,Diploma at level 3,FT,DEG,Positive
4,STU56635,31-40,White,Female,White,Yes,Quintile 2,Quintile 2,Quintile 2,Quintile 2,...,Any_other_religion_or_belief,Refused_Unknown_or_not_collected,No,Mental health condition,Disability_Reported,Care_experienced,Level 3 qualifications of which some are subje...,FT,DEG,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,STU64519,20,White,Female,White,Yes,Quintile 3,Quintile 3,Quintile 3,Quintile 1,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,A/AS level,FT,DEG,Positive
742,STU83875,41-50,White,Female,White,Yes,Quintile 2,Quintile 2,Quintile 2,Quintile 1,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Higher National Diploma (HND),FT,DEG,Negative
743,STU66401,21-25,White,Female,White,Yes,Quintile 1,Quintile 1,Quintile 1,Quintile 1,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Level 3 qualifications of which none are subje...,FT,DEG,Positive
744,STU91604,31-40,White,Female,White,Yes,Quintile 5,Quintile 4,Quintile 5,Quintile 5,...,Christian,Heterosexual,No,No disability reported,No_Disability_Reported,Care_experienced,Higher National Diploma (HND),FT,DEG,Positive


**Selecting relevant columns from the below dataframe**

In [91]:
concat_hist_char_f = concat_hist_char_f[['StudentNumber', 'age_on_entry', 'ethnicity_16_groups', 'sex',
       'adult_he_2011_quintile_desc', 'imd_desc',
       'idaci_desc',  'disability_type', 'IPLEVEL','IPSTARTMODE',
       'Outcome']]

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 93, Finished, Available, Finished)

**Creating copy of the below dataframe and saving in another dataframe**

In [92]:
concat_hist_char_export_f = concat_hist_char_f.copy()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 94, Finished, Available, Finished)

**Creating concatenated column by joining characteristics in same order as above to map with external progression data**

In [93]:
# Concatenate columns by converting all values to strings
concat_hist_char_f['Concatenated_column'] = (
    concat_hist_char_f['adult_he_2011_quintile_desc'].astype(str) + 
    concat_hist_char_f['age_on_entry'].astype(str) + 
    concat_hist_char_f['disability_type'].astype(str) + 
    concat_hist_char_f['ethnicity_16_groups'].astype(str) + 
    concat_hist_char_f['idaci_desc'].astype(str) + 
    concat_hist_char_f['imd_desc'].astype(str) + 
    concat_hist_char_f['IPSTARTMODE'].astype(str) + 
    concat_hist_char_f['sex'].astype(str)
)

#renaming col name and slecting some columns 
concat_hist_char_f.rename(columns={'Outcome': 'Actual_Outcome'}, inplace=True)
concat_hist_char_f=concat_hist_char_f[['StudentNumber','IPLEVEL','Actual_Outcome','Concatenated_column']]
concat_hist_char_f

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 95, Finished, Available, Finished)

,StudentNumber,IPLEVEL,Actual_Outcome,Concatenated_column
0,STU69375,DEG,Negative,"Quintile 241-50Sensory, medical or physical im..."
1,STU68962,DEG,Negative,Quintile 541-50Mental health conditionWhiteQui...
2,STU65338,DEG,Positive,Quintile 431-40Mental health conditionWhiteQui...
3,STU62316,DEG,Positive,Quintile 126-30Mental health conditionWhiteQui...
4,STU56635,DEG,Positive,Quintile 231-40Mental health conditionWhiteQui...
...,...,...,...,...
741,STU64519,DEG,Positive,Quintile 320No disability reportedWhiteQuintil...
742,STU83875,DEG,Negative,Quintile 241-50No disability reportedWhiteQuin...
743,STU66401,DEG,Positive,Quintile 121-25No disability reportedWhiteQuin...
744,STU91604,DEG,Positive,Quintile 431-40No disability reportedWhiteQuin...


In [94]:
#dropping duplicates from the dataframe
df_ft1 = df_ft1.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 96, Finished, Available, Finished)

**Inner joining with historical Arden data with external progression data on concatenated column**

In [95]:
joined_df_f = df_ft1.merge(concat_hist_char_f,on='Concatenated_column', how='inner')
joined_df_f = joined_df_f.drop_duplicates()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 97, Finished, Available, Finished)

In [96]:
#displaying the dataframe
joined_df_f

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 98, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column,StudentNumber,IPLEVEL,Actual_Outcome
0,0.710000,1,Quintile 321-25Cognitive or learning difficult...,STU62768,DEG,Negative
1,0.740000,1,Quintile 521-25Cognitive or learning difficult...,STU62566,DEG,Positive
2,0.730000,1,"Quintile 321-25Sensory, medical or physical im...",STU56459,DEG,Positive
3,0.690000,1,"Quintile 521-25Sensory, medical or physical im...",STU56204,DEG,Negative
4,0.590000,1,Quintile 121-25Mental health conditionWhiteQui...,STU66660,DEG,Positive
...,...,...,...,...,...,...
687,0.820000,3,Quintile 331-40No disability reportedWhiteQuin...,STU87073,DEG,Positive
688,0.820000,3,Quintile 431-40No disability reportedBlack or ...,STU70051,DEG,Positive
689,0.820000,3,Quintile 231-40No disability reportedWhiteQuin...,STU58231,DEG,Positive
690,0.820000,3,Quintile 131-40No disability reportedAsian or ...,STU56997,DEG,Negative


**Applying inner join to tag predicted outcome rate and derive quintile to each student**

In [97]:
export_ft_df_f = pd.merge(joined_df_f,concat_hist_char_export_f,on='StudentNumber',how = 'inner')

export_ft_df_f.drop(['IPLEVEL_y','Outcome'], axis=1, inplace=True)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 99, Finished, Available, Finished)

**Creating copy of the dataframe and saving in another dataframe**

In [98]:
raw_data_ft1=export_ft_df_f.copy()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 100, Finished, Available, Finished)

In [99]:
#displaying the dataframe
raw_data_ft1

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 101, Finished, Available, Finished)

,Predicted_outcome_rate,Derived_progression_quintile,Concatenated_column,StudentNumber,IPLEVEL_x,Actual_Outcome,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPSTARTMODE
0,0.710000,1,Quintile 321-25Cognitive or learning difficult...,STU62768,DEG,Negative,21-25,Black or black British - African,Female,Quintile 3,Quintile 1,Quintile 1,Cognitive or learning difficulties,FT
1,0.740000,1,Quintile 521-25Cognitive or learning difficult...,STU62566,DEG,Positive,21-25,Black or black British - African,Female,Quintile 5,Quintile 1,Quintile 1,Cognitive or learning difficulties,FT
2,0.730000,1,"Quintile 321-25Sensory, medical or physical im...",STU56459,DEG,Positive,21-25,White,Female,Quintile 3,Quintile 2,Quintile 2,"Sensory, medical or physical impairments",FT
3,0.690000,1,"Quintile 521-25Sensory, medical or physical im...",STU56204,DEG,Negative,21-25,Black or black British - African,Female,Quintile 5,Quintile 3,Quintile 3,"Sensory, medical or physical impairments",FT
4,0.590000,1,Quintile 121-25Mental health conditionWhiteQui...,STU66660,DEG,Positive,21-25,White,Female,Quintile 1,Quintile 1,Quintile 1,Mental health condition,FT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,0.820000,3,Quintile 331-40No disability reportedWhiteQuin...,STU87073,DEG,Positive,31-40,White,Male,Quintile 3,Quintile 2,Quintile 2,No disability reported,FT
690,0.820000,3,Quintile 431-40No disability reportedBlack or ...,STU70051,DEG,Positive,31-40,Black or black British - African,Male,Quintile 4,Quintile 2,Quintile 1,No disability reported,FT
691,0.820000,3,Quintile 231-40No disability reportedWhiteQuin...,STU58231,DEG,Positive,31-40,White,Male,Quintile 2,Quintile 2,Quintile 1,No disability reported,FT
692,0.820000,3,Quintile 131-40No disability reportedAsian or ...,STU56997,DEG,Negative,31-40,Asian or Asian British - Bangladeshi,Male,Quintile 1,Quintile 1,Quintile 1,No disability reported,FT


**Combining PT and FT data**

In [100]:
# Concatenate the DataFrames 
combined_df = pd.concat([raw_data_pt, raw_data_ft1], ignore_index=True) 
print(f"Combined DataFrame shape: {combined_df.shape}")

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 102, Finished, Available, Finished)

Combined DataFrame shape: (1380, 14)


In [101]:
combined_df

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 103, Finished, Available, Finished)

,StudentNumber,IPLEVEL_x,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPSTARTMODE,Concatenated_column,Actual_Outcome,Predicted_outcome_rate,Derived_progression_quintile
0,STU59079,PGTM,21-25,White,Female,Quintile 3,Quintile 2,Quintile 4,"Sensory, medical or physical impairments",PT,"Quintile 321-25Sensory, medical or physical im...",Positive,0.730000,1
1,STU59158,PGTM,21-25,White,Female,Quintile 2,Quintile 3,Quintile 4,"Sensory, medical or physical impairments",PT,"Quintile 221-25Sensory, medical or physical im...",Negative,0.750000,1
2,STU69756,PGTM,21-25,White,Female,Quintile 3,Quintile 2,Quintile 2,Social or communication impairment,PT,Quintile 321-25Social or communication impairm...,Positive,0.660000,1
3,STU56579,DEG,21-25,White,Female,Quintile 1,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 121-25Mental health conditionWhiteQui...,Negative,0.600000,1
4,STU74215,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 2,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 221-25Mental health conditionAsian or...,Negative,0.530000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,STU87073,DEG,31-40,White,Male,Quintile 3,Quintile 2,Quintile 2,No disability reported,FT,Quintile 331-40No disability reportedWhiteQuin...,Positive,0.820000,3
1376,STU70051,DEG,31-40,Black or black British - African,Male,Quintile 4,Quintile 2,Quintile 1,No disability reported,FT,Quintile 431-40No disability reportedBlack or ...,Positive,0.820000,3
1377,STU58231,DEG,31-40,White,Male,Quintile 2,Quintile 2,Quintile 1,No disability reported,FT,Quintile 231-40No disability reportedWhiteQuin...,Positive,0.820000,3
1378,STU56997,DEG,31-40,Asian or Asian British - Bangladeshi,Male,Quintile 1,Quintile 1,Quintile 1,No disability reported,FT,Quintile 131-40No disability reportedAsian or ...,Negative,0.820000,3


**Sorting the data in ascending by 'Predicted_outcome_rate'**

In [102]:
dfp1_sorted=combined_df.sort_values(by='Predicted_outcome_rate')
dfp1_sorted

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 104, Finished, Available, Finished)

,StudentNumber,IPLEVEL_x,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPSTARTMODE,Concatenated_column,Actual_Outcome,Predicted_outcome_rate,Derived_progression_quintile
533,STU96605,OPGT,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 3,Quintile 3,Quintile 3,No disability reported,PT,Quintile 331-40No disability reportedAsian or ...,Positive,0.490000,1
4,STU74215,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 2,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 221-25Mental health conditionAsian or...,Negative,0.530000,1
535,STU97006,OPGT,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 5,Quintile 3,Quintile 3,No disability reported,PT,Quintile 531-40No disability reportedAsian or ...,Positive,0.530000,1
537,STU48198,OUG,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 1,Quintile 4,Quintile 4,No disability reported,PT,Quintile 131-40No disability reportedAsian or ...,Negative,0.550000,1
1172,STU57272,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 1,Quintile 1,Quintile 2,No disability reported,FT,Quintile 121-25No disability reportedAsian or ...,Negative,0.550000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,STU66494,DEG,51+,Black or black British - African,Female,Quintile 5,Quintile 3,Quintile 5,"Sensory, medical or physical impairments",FT,"Quintile 551+Sensory, medical or physical impa...",Negative,0.940000,5
181,STU50546,PGTM,26-30,Black or black British - African,Male,Quintile 5,Quintile 5,Quintile 5,No disability reported,PT,Quintile 526-30No disability reportedBlack or ...,Positive,0.940000,5
901,STU59347,DEG,51+,Black or black British - African,Male,Quintile 3,Quintile 4,Quintile 5,No disability reported,FT,Quintile 351+No disability reportedBlack or bl...,Positive,0.950000,5
889,STU68299,DEG,51+,Black or black British - African,Female,Quintile 5,Quintile 5,Quintile 5,No disability reported,FT,Quintile 551+No disability reportedBlack or bl...,Positive,0.950000,5


**We observed the categories of features influencing our positive and negative progression classes by sorting the progression rates in ascending order and analyzing which categories of characteristics have the greatest impact on either a negative or positive progression rate:**

Top 30- top 100-top 200: 

    Adult HE Quintile: Quintile 1 has greater impact towards negative progression rate.

    IMD: Quintile 1 has greater impact towards negative progression rate.

	IDACI: Quintile 1 has greater impact towards negative progression rate.

	Age: '21-25’ has greater impact towards negative progression rate.

Bottom 30-Bottom 100-Bottom 200:

	Adult HE Quintile: Quintile 5 has greater impact towards positive progression rate.

	IMD: Quintile 5 has greater impact towards positive progression rate.

	IDACI: Quintile 5 has greater impact towards positive progression rate.

	Age: '51+’ has greater impact towards positive progression rate.

<mark>TOP30</mark>

**Selecting top 30 rows from the dataframe**

In [103]:
dfp1_sorted_top30=dfp1_sorted.head(30)
len(dfp1_sorted_top30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 105, Finished, Available, Finished)

30

**Value counts for 'age_on_entry' column**

In [104]:
age_on_entry_value_counts = dfp1_sorted_top30['age_on_entry'].value_counts()
print(age_on_entry_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 106, Finished, Available, Finished)

age_on_entry
21-25    15
31-40    11
26-30     2
41-50     2
Name: count, dtype: int64


**Value counts for 'ethnicity_16_groups' column**

In [105]:
ethnicity_16_groups_value_counts = dfp1_sorted_top30['ethnicity_16_groups'].value_counts()
print(ethnicity_16_groups_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 107, Finished, Available, Finished)

ethnicity_16_groups
Asian or Asian British - Pakistani      13
Other ethnic group                       4
Black or black British - African         4
Asian or Asian British - Bangladeshi     3
Mixed - white and black African          2
White                                    2
Black or black British - Caribbean       1
Mixed - white and Asian                  1
Name: count, dtype: int64


**Value counts for 'sex' column**

In [106]:
sex_value_counts = dfp1_sorted_top30['sex'].value_counts()
print(sex_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 108, Finished, Available, Finished)

sex
Female    19
Male      11
Name: count, dtype: int64


**Value counts for 'adult_he_2011_quintile_desc' column**

In [107]:
adult_he_2011_quintile_desc_value_counts = dfp1_sorted_top30['adult_he_2011_quintile_desc'].value_counts()
print(adult_he_2011_quintile_desc_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 109, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 1    14
Quintile 2     8
Quintile 5     4
Quintile 3     3
Quintile 4     1
Name: count, dtype: int64


**Value counts for 'imd_desc' column**

In [108]:
imd_desc_value_counts = dfp1_sorted_top30['imd_desc'].value_counts()
print(imd_desc_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 110, Finished, Available, Finished)

imd_desc
Quintile 1    16
Quintile 3     5
Quintile 2     5
Quintile 4     2
Quintile 5     2
Name: count, dtype: int64


**Value counts for 'idaci_desc' column**

In [109]:
idaci_desc_value_counts = dfp1_sorted_top30['idaci_desc'].value_counts()
print(idaci_desc_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 111, Finished, Available, Finished)

idaci_desc
Quintile 1    15
Quintile 3     4
Quintile 2     4
Quintile 5     4
Quintile 4     3
Name: count, dtype: int64


**Value counts for 'disability_type' column**

In [110]:

disability_type_value_counts = dfp1_sorted_top30['disability_type'].value_counts()
print(disability_type_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 112, Finished, Available, Finished)

disability_type
No disability reported                      22
Mental health condition                      5
Cognitive or learning difficulties           1
Sensory, medical or physical impairments     1
Multiple or other impairments                1
Name: count, dtype: int64


**Value counts for 'iplevel' column**

In [111]:
IPLEVEL_x_value_counts = dfp1_sorted_top30['IPLEVEL_x'].value_counts()
print(IPLEVEL_x_value_counts)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 113, Finished, Available, Finished)

IPLEVEL_x
DEG     22
PGTM     4
OPGT     3
OUG      1
Name: count, dtype: int64


<mark>**BOTTOM 30**</mark>

**Selecting bottom 30 rows**

In [112]:
dfp1_sorted_bottom30=dfp1_sorted.tail(30)
len(dfp1_sorted_bottom30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 114, Finished, Available, Finished)

30

**Value counts for 'iplevel' column**

In [113]:
IPLEVEL_x_value_counts_b30 = dfp1_sorted_bottom30['IPLEVEL_x'].value_counts()
print(IPLEVEL_x_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 115, Finished, Available, Finished)

IPLEVEL_x
DEG     20
PGTM     9
OPGT     1
Name: count, dtype: int64


**Value counts for 'age_on_entry' column**

In [114]:
age_on_entry_value_counts_b30 = dfp1_sorted_bottom30['age_on_entry'].value_counts()
print(age_on_entry_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 116, Finished, Available, Finished)

age_on_entry
51+      13
41-50     8
31-40     5
26-30     4
Name: count, dtype: int64


**Value counts for 'ethnicity_16_groups' column**

In [115]:
ethnicity_16_groups_value_counts_b30 = dfp1_sorted_bottom30['ethnicity_16_groups'].value_counts()
print(ethnicity_16_groups_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 117, Finished, Available, Finished)

ethnicity_16_groups
Black or black British - African        14
White                                    6
Black or black British - Caribbean       2
Mixed - white and black Caribbean        2
Asian or Asian British - Indian          2
Mixed - white and black African          2
Refused or unknown                       1
Asian or Asian British - Bangladeshi     1
Name: count, dtype: int64


**Value counts for 'sex' column**

In [116]:
sex_value_counts_b30 = dfp1_sorted_bottom30['sex'].value_counts()
print(sex_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 118, Finished, Available, Finished)

sex
Male      15
Female    15
Name: count, dtype: int64


**Value counts for 'adult_he_2011_quintile_desc' column**

In [117]:
adult_he_2011_quintile_desc_value_counts_b30 = dfp1_sorted_bottom30['adult_he_2011_quintile_desc'].value_counts()
print(adult_he_2011_quintile_desc_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 119, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 5    20
Quintile 4     7
Quintile 1     1
Quintile 2     1
Quintile 3     1
Name: count, dtype: int64


**Value counts for 'imd_desc' column**

In [118]:
imd_desc_value_counts_b30 = dfp1_sorted_bottom30['imd_desc'].value_counts()
print(imd_desc_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 120, Finished, Available, Finished)

imd_desc
Quintile 5    15
Quintile 4     6
Quintile 3     3
Quintile 2     3
Quintile 1     2
               1
Name: count, dtype: int64


**Value counts for 'idaci_desc' column**

In [119]:
idaci_desc_value_counts_b30 = dfp1_sorted_bottom30['idaci_desc'].value_counts()
print(idaci_desc_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 121, Finished, Available, Finished)

idaci_desc
Quintile 5    19
Quintile 1     4
Quintile 3     3
Quintile 4     3
               1
Name: count, dtype: int64


**Value counts for 'disability_type' column**

In [120]:
disability_type_value_counts_b30 = dfp1_sorted_bottom30['disability_type'].value_counts()
print(disability_type_value_counts_b30)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 122, Finished, Available, Finished)

disability_type
No disability reported                      25
Sensory, medical or physical impairments     3
Cognitive or learning difficulties           2
Name: count, dtype: int64


<mark> TOP 100 </mark>

**Selecting top 100 rows**

In [121]:
dfp1_sorted_top100=dfp1_sorted.head(100)
dfp1_sorted_top100

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 123, Finished, Available, Finished)

,StudentNumber,IPLEVEL_x,age_on_entry,ethnicity_16_groups,sex,adult_he_2011_quintile_desc,imd_desc,idaci_desc,disability_type,IPSTARTMODE,Concatenated_column,Actual_Outcome,Predicted_outcome_rate,Derived_progression_quintile
533,STU96605,OPGT,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 3,Quintile 3,Quintile 3,No disability reported,PT,Quintile 331-40No disability reportedAsian or ...,Positive,0.490000,1
4,STU74215,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 2,Quintile 1,Quintile 1,Mental health condition,PT,Quintile 221-25Mental health conditionAsian or...,Negative,0.530000,1
535,STU97006,OPGT,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 5,Quintile 3,Quintile 3,No disability reported,PT,Quintile 531-40No disability reportedAsian or ...,Positive,0.530000,1
537,STU48198,OUG,31-40,Asian or Asian British - Bangladeshi,Female,Quintile 1,Quintile 4,Quintile 4,No disability reported,PT,Quintile 131-40No disability reportedAsian or ...,Negative,0.550000,1
1172,STU57272,DEG,21-25,Asian or Asian British - Pakistani,Female,Quintile 1,Quintile 1,Quintile 2,No disability reported,FT,Quintile 121-25No disability reportedAsian or ...,Negative,0.550000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,STU57541,DEG,21-25,Asian or Asian British - Pakistani,Male,Quintile 2,Quintile 2,Quintile 1,No disability reported,FT,Quintile 221-25No disability reportedAsian or ...,Positive,0.690000,1
1235,STU66721,DEG,31-40,Asian or Asian British - Pakistani,Female,Quintile 2,Quintile 2,Quintile 2,No disability reported,FT,Quintile 231-40No disability reportedAsian or ...,Negative,0.690000,1
36,STU83187,PGTM,41-50,White,Female,Quintile 3,Quintile 5,Quintile 5,Mental health condition,PT,Quintile 341-50Mental health conditionWhiteQui...,Positive,0.690000,1
1247,STU64166,DEG,31-40,Black or black British - Caribbean,Female,Quintile 1,Quintile 1,Quintile 1,No disability reported,FT,Quintile 131-40No disability reportedBlack or ...,Negative,0.690000,1


**Columns present in the dataframe**

In [122]:
dfp1_sorted_top100.columns

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 124, Finished, Available, Finished)

Index(['StudentNumber', 'IPLEVEL_x', 'age_on_entry', 'ethnicity_16_groups',
       'sex', 'adult_he_2011_quintile_desc', 'imd_desc', 'idaci_desc',
       'disability_type', 'IPSTARTMODE', 'Concatenated_column',
       'Actual_Outcome', 'Predicted_outcome_rate',
       'Derived_progression_quintile'],
      dtype='object')

**Value counts for 'age_on_entry' column**

In [123]:
dfp1_sorted_top100['age_on_entry'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 125, Finished, Available, Finished)

age_on_entry
21-25    39
31-40    36
41-50    13
26-30    12
Name: count, dtype: int64

**Value counts for 'ethnicity_16_groups' column**

In [124]:
dfp1_sorted_top100['ethnicity_16_groups'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 126, Finished, Available, Finished)

ethnicity_16_groups
Asian or Asian British - Pakistani      26
Black or black British - African        24
White                                   22
Black or black British - Caribbean       7
Other ethnic group                       6
Asian or Asian British - Bangladeshi     5
Asian or Asian British - Indian          3
Mixed - white and black African          2
Refused or unknown                       2
Asian or Asian British - other           2
Mixed - white and Asian                  1
Name: count, dtype: int64

**Value counts for 'sex' column**

In [125]:
dfp1_sorted_top100['sex'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 127, Finished, Available, Finished)

sex
Female    79
Male      21
Name: count, dtype: int64

**Value counts for 'adult_he_2011_quintile_desc' column**

In [126]:
dfp1_sorted_top100['adult_he_2011_quintile_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 128, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 1    41
Quintile 2    27
Quintile 3    12
Quintile 5    10
Quintile 4     9
               1
Name: count, dtype: int64

**Value counts for 'imd_desc' column**

In [127]:
dfp1_sorted_top100['imd_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 129, Finished, Available, Finished)

imd_desc
Quintile 1    49
Quintile 3    20
Quintile 2    18
Quintile 4     7
Quintile 5     5
               1
Name: count, dtype: int64

**Value counts for 'idaci_desc' column**

In [128]:
dfp1_sorted_top100['idaci_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 130, Finished, Available, Finished)

idaci_desc
Quintile 1    44
Quintile 3    20
Quintile 2    17
Quintile 4    11
Quintile 5     7
               1
Name: count, dtype: int64

**Value counts for 'disability_desc' column**

In [129]:
dfp1_sorted_top100['disability_type'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 131, Finished, Available, Finished)

disability_type
No disability reported                      67
Mental health condition                     21
Sensory, medical or physical impairments     5
Multiple or other impairments                3
Social or communication impairment           3
Cognitive or learning difficulties           1
Name: count, dtype: int64

**Value counts for 'IPLEVEL_x' column**

In [130]:
dfp1_sorted_top100['IPLEVEL_x'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 132, Finished, Available, Finished)

IPLEVEL_x
DEG     70
PGTM    24
OPGT     4
OUG      2
Name: count, dtype: int64

<mark>**Bottom 100**</mark>

**Selecting bottom 100 rows**

In [131]:
dfp1_sorted_bottom100=dfp1_sorted.tail(100)
len(dfp1_sorted_bottom100)

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 133, Finished, Available, Finished)

100

**Value counts for 'IPLEVEL_x' column**

In [132]:
dfp1_sorted_bottom100['IPLEVEL_x'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 134, Finished, Available, Finished)

IPLEVEL_x
DEG     73
PGTM    19
OUG      5
OPGT     2
PUGO     1
Name: count, dtype: int64

**Value counts for 'age_on_entry' column**

In [133]:
dfp1_sorted_bottom100['age_on_entry'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 135, Finished, Available, Finished)

age_on_entry
51+      37
41-50    27
26-30    21
31-40    12
21-25     3
Name: count, dtype: int64

**Value counts for 'ethnicity_16_groups' column**

In [134]:
dfp1_sorted_bottom100['ethnicity_16_groups'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 136, Finished, Available, Finished)

ethnicity_16_groups
White                                   35
Black or black British - African        26
Refused or unknown                      15
Asian or Asian British - Indian          7
Black or black British - Caribbean       7
Asian or Asian British - Bangladeshi     3
Asian or Asian British - Pakistani       2
Mixed - white and black Caribbean        2
Mixed - white and black African          2
Other ethnic group                       1
Name: count, dtype: int64

**Value counts for 'sex' column**

In [135]:
dfp1_sorted_bottom100['sex'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 137, Finished, Available, Finished)

sex
Male      62
Female    38
Name: count, dtype: int64

**Value counts for 'adult_he_2011_quintile_desc' column**

In [136]:
dfp1_sorted_bottom100['adult_he_2011_quintile_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 138, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 5    61
Quintile 4    25
Quintile 3     6
Quintile 2     6
Quintile 1     2
Name: count, dtype: int64

**Value counts for 'imd_desc' column**

In [137]:
dfp1_sorted_bottom100['imd_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 139, Finished, Available, Finished)

imd_desc
Quintile 5    37
Quintile 4    25
Quintile 2    20
Quintile 3    13
Quintile 1     4
               1
Name: count, dtype: int64

**Value counts for 'idaci_desc' column**

In [138]:
dfp1_sorted_bottom100['idaci_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 140, Finished, Available, Finished)

idaci_desc
Quintile 5    43
Quintile 2    19
Quintile 4    15
Quintile 3    11
Quintile 1    11
               1
Name: count, dtype: int64

**Value counts for 'disability_type' column**

In [139]:
dfp1_sorted_bottom100['disability_type'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 141, Finished, Available, Finished)

disability_type
No disability reported                      91
Sensory, medical or physical impairments     5
Cognitive or learning difficulties           3
Multiple or other impairments                1
Name: count, dtype: int64

<mark>**TOP 200**</mark>

**Selecting top 200 rows**

In [140]:
dfp1_sorted_top200=dfp1_sorted.head(200)
dfp1_sorted_top200.columns

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 142, Finished, Available, Finished)

Index(['StudentNumber', 'IPLEVEL_x', 'age_on_entry', 'ethnicity_16_groups',
       'sex', 'adult_he_2011_quintile_desc', 'imd_desc', 'idaci_desc',
       'disability_type', 'IPSTARTMODE', 'Concatenated_column',
       'Actual_Outcome', 'Predicted_outcome_rate',
       'Derived_progression_quintile'],
      dtype='object')

**Value counts for 'age_on_entry' column**

In [141]:
dfp1_sorted_top200['age_on_entry'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 143, Finished, Available, Finished)

age_on_entry
31-40    72
21-25    71
41-50    36
26-30    21
Name: count, dtype: int64

**Value counts for 'ethnicity_16_groups' column**

In [142]:
dfp1_sorted_top200['ethnicity_16_groups'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 144, Finished, Available, Finished)

ethnicity_16_groups
White                                   62
Black or black British - African        56
Asian or Asian British - Pakistani      34
Black or black British - Caribbean      17
Other ethnic group                       8
Asian or Asian British - Bangladeshi     5
Asian or Asian British - Indian          5
Asian or Asian British - other           4
Mixed - white and black African          3
Refused or unknown                       3
Mixed - white and black Caribbean        2
Mixed - white and Asian                  1
Name: count, dtype: int64

**Value counts for 'sex' column**

In [143]:
dfp1_sorted_top200['sex'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 145, Finished, Available, Finished)

sex
Female    164
Male       36
Name: count, dtype: int64

**Value counts for 'adult_he_2011_quintile_desc' column**

In [144]:
dfp1_sorted_top200['adult_he_2011_quintile_desc'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 146, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 1    74
Quintile 2    55
Quintile 3    30
Quintile 4    22
Quintile 5    17
               2
Name: count, dtype: int64

**Value counts for 'imd_desc' column**

In [145]:
dfp1_sorted_top200['imd_desc'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 147, Finished, Available, Finished)

imd_desc
Quintile 1    99
Quintile 2    44
Quintile 3    34
Quintile 4    11
Quintile 5     7
               5
Name: count, dtype: int64

**Value counts for 'idaci_desc' column**

In [146]:
dfp1_sorted_top200['idaci_desc'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 148, Finished, Available, Finished)

idaci_desc
Quintile 1    95
Quintile 3    43
Quintile 2    33
Quintile 4    16
Quintile 5     8
               5
Name: count, dtype: int64

**Value counts for 'disability_type' column**

In [147]:
dfp1_sorted_top200['disability_type'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 149, Finished, Available, Finished)

disability_type
No disability reported                      150
Mental health condition                      28
Sensory, medical or physical impairments     11
Multiple or other impairments                 5
Cognitive or learning difficulties            3
Social or communication impairment            3
Name: count, dtype: int64

**<mark>Bottom 200</mark>**

**Selecting bottom 200 rows**

In [148]:
dfp1_sorted_bottom200=dfp1_sorted.tail(200)
dfp1_sorted_bottom200.columns

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 150, Finished, Available, Finished)

Index(['StudentNumber', 'IPLEVEL_x', 'age_on_entry', 'ethnicity_16_groups',
       'sex', 'adult_he_2011_quintile_desc', 'imd_desc', 'idaci_desc',
       'disability_type', 'IPSTARTMODE', 'Concatenated_column',
       'Actual_Outcome', 'Predicted_outcome_rate',
       'Derived_progression_quintile'],
      dtype='object')

**Value counts for 'age_on_entry' column**

In [149]:
dfp1_sorted_bottom200['age_on_entry'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 151, Finished, Available, Finished)

age_on_entry
51+      71
41-50    49
26-30    41
31-40    31
21-25     8
Name: count, dtype: int64

**Value counts for 'ethnicity_16_groups' column**

In [150]:
dfp1_sorted_bottom200['ethnicity_16_groups'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 152, Finished, Available, Finished)

ethnicity_16_groups
White                                   90
Black or black British - African        54
Refused or unknown                      19
Black or black British - Caribbean      12
Asian or Asian British - Indian          8
Asian or Asian British - Bangladeshi     7
Asian or Asian British - Pakistani       3
Other ethnic group                       2
Mixed - white and black Caribbean        2
Mixed - white and black African          2
Asian or Asian British - other           1
Name: count, dtype: int64

**Value counts for 'sex' column**

In [151]:
dfp1_sorted_bottom200['sex'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 153, Finished, Available, Finished)

sex
Male      119
Female     81
Name: count, dtype: int64

**Value counts for 'adult_he_2011_quintile_desc' column**

In [152]:
dfp1_sorted_bottom200['adult_he_2011_quintile_desc'].value_counts()

StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 154, Finished, Available, Finished)

adult_he_2011_quintile_desc
Quintile 5    107
Quintile 4     58
Quintile 3     18
Quintile 2     10
Quintile 1      6
                1
Name: count, dtype: int64

**Value counts for 'imd_desc' column**

In [153]:
dfp1_sorted_bottom200['imd_desc'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 155, Finished, Available, Finished)

imd_desc
Quintile 2    50
Quintile 5    48
Quintile 4    48
Quintile 3    37
Quintile 1    14
               3
Name: count, dtype: int64

**Value counts for 'idaci_desc' column**

In [154]:
dfp1_sorted_bottom200['idaci_desc'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 156, Finished, Available, Finished)

idaci_desc
Quintile 5    58
Quintile 2    46
Quintile 4    42
Quintile 1    36
Quintile 3    15
               3
Name: count, dtype: int64

**Value counts for 'disability_type' column**

In [155]:
dfp1_sorted_bottom200['disability_type'].value_counts()


StatementMeta(, c3362933-4b99-4fc4-b98a-e8b07e6c5a39, 157, Finished, Available, Finished)

disability_type
No disability reported                      183
Sensory, medical or physical impairments     10
Cognitive or learning difficulties            6
Multiple or other impairments                 1
Name: count, dtype: int64